# Using the Vertex AI PaLM API to explain BQML Clustering - Our Very Own Persona Builder

# Core code
Let's define some variables that will be used throughout this notebook.

These are the GCP Project ID `project_id`, the Model name `model_name` which is any name you prefer, and finally the Dataset name `dataset_name`.
The dataset needs to exist in the same Project as `project_id` and you'll need appropriate access to create and delete.

## Setup

In [1]:
from typing import Union
import sys

import os
import io
import json
import base64
import requests
import concurrent.futures
import time

import numpy as np
import pandas as pd

import vertexai.preview.language_models
from google.cloud import aiplatform
from google.cloud import documentai
from google.cloud.documentai_v1 import Document
from google.cloud import storage
from google.cloud import bigquery

from IPython.display import display, Markdown, Latex


In [2]:
PROJECT_ID = 'mg-ce-demos'
REGION = 'us-central1'


In [3]:
# vertex ai clients
vertexai.init(project = PROJECT_ID, location = REGION)
aiplatform.init(project = PROJECT_ID, location = REGION)


## Ideate with Vertex AI PaLM API

In [11]:
textgen_model = vertexai.preview.language_models.TextGenerationModel.from_pretrained('text-bison@latest')

### Now, we send our prompt to Google GenAI API for some LLM magic

In [16]:
# Provide some context
pre_text = "Pretend you're a marketing specialist, "

# Ask the LLM
text = "Generate a marketing campaign for the launch of a new condominium community in Cincinnati, OH"

prompt = pre_text+text

In [17]:
display(Markdown('## Prompt:'))
print(prompt)
display(Markdown('## Response:'))

# Send prompt to LLM
display(Markdown(str(textgen_model.predict(
   (prompt),
    max_output_tokens=1024,
    temperature=0.4,
    top_p=0.8,
    top_k=40,
))))

## Prompt:

Pretend you're a marketing specialist, Generate a marketing campaign for the launch of a new condominium community in Cincinnati, OH


## Response:

**Cincinnati's newest condominium community is now open!**

**The Residences at 101 Broadway** is a luxury condominium community located in the heart of downtown Cincinnati. With its prime location, stunning architecture, and state-of-the-art amenities, The Residences at 101 Broadway is the perfect place to call home.

**Key features of The Residences at 101 Broadway include:**

* **Prime location:** The Residences at 101 Broadway is located just steps from Cincinnati's Central Business District, the Cincinnati Art Museum, and the Cincinnati Symphony Orchestra.
* **Stunning architecture:** The Residences at 101 Broadway features stunning architecture designed by award-winning architect, Michael Graves.
* **State-of-the-art amenities:** The Residences at 101 Broadway offers a variety of state-of-the-art amenities, including a rooftop pool, a fitness center, and a business center.

**To celebrate the launch of The Residences at 101 Broadway, we are offering special introductory pricing and incentives.**

**Contact us today to learn more and schedule a tour!**

**Marketing Campaign**

**Social Media:**

* **Facebook:** Post regular updates about The Residences at 101 Broadway on Facebook. Share photos of the community, upcoming events, and special offers.
* **Twitter:** Tweet about The Residences at 101 Broadway and use relevant hashtags to reach a wider audience.
* **Instagram:** Share stunning photos of The Residences at 101 Broadway on Instagram. Use relevant hashtags to attract potential buyers.

**Email:**

* Send out an email newsletter to your list of contacts announcing the launch of The Residences at 101 Broadway. Include information about the community, special pricing, and incentives.
* Create an email drip campaign to nurture leads and encourage them to take action.

**Paid Advertising:**

* Run paid ads on Google and social media to target potential buyers.
* Use targeted keywords and phrases to reach people who are interested in buying a condominium in Cincinnati.

**Public Relations:**

* Send out a press release announcing the launch of The Residences at 101 Broadway.
* Get featured in local publications and websites.
* Partner with local businesses to host events and giveaways.

**Outdoor Advertising:**

* Place billboards and bus ads around the city to promote The Residences at 101 Broadway.
* Run print ads in local newspapers and magazines.

**By following these marketing tips, you can help to ensure the success of The Residences at 101 Broadway.**

Voila! We've now used k-means clustering to create groups of spenders and explain their profiles.

Sometimes, though, you want a little bit [extra](https://cloud.google.com/blog/transform/prompt-debunking-five-generative-ai-misconceptions).

In [22]:
def marketing_bot(text):
    pre_text = "Pretend you're a marketing specialist, "
    prompt = pre_text + text
    result = textgen_model.predict(
        prompt,
        max_output_tokens=1024,
        temperature=0.4,
        top_p=0.8,
        top_k=40,
    )
    
    return prompt, result

A beautiful mountain range with snow-capped peaks and a river running through it. The sun is shining and the sky is blue.  Photorealistic

come up with creative brand persona for each that includes the detail of their favorite movie, a summary of how this relates to their purchasing behavior, and a witty e-mail headline for marketing campaign targeted to their group

## Now we can stick it behind a UI

In [23]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown(
    """
    ## Marketing Bot

    ### Enter a task...

    """)
    with gr.Row():
      with gr.Column():
        input_text = gr.Textbox(label="Task", placeholder="Pretend you're a marketing specialist, ")
        
        
    with gr.Row():
      generate = gr.Button("Generate Response")

    with gr.Row():
      label2 = gr.Textbox(label="Prompt")
    with gr.Row():
      label3 = gr.Textbox(label="Response generated by LLM")

    generate.click(marketing_bot,input_text, [label2, label3])
demo.launch(share=False, debug=False)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
